<a href="https://colab.research.google.com/github/castrokelly/Data-Science/blob/main/Abordagem_Ensemble_para_Detec%C3%A7%C3%A3o_e_Classifica%C3%A7%C3%A3o_de_Anomalias_em_Po%C3%A7os_de_Petr%C3%B3leo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abordagem Ensemble para Detecção e Classificação de Anomalias em Poços de Petróleo: um Estudo Aplicado ao Dataset 3W

**Centro de Ciências Matemáticas Aplicadas à Indústria (CeMEAI)**</br>
**Instituto de Ciências Matemáticas e de Computação (ICMC)**</br>
**Universidade de São Paulo**</br>
</br>
Aluna: **Kelly Christine Alvarenga de Castro**
Área de concentração: Ciências de Dados
Orientador: **Prof. Dr. Cláudio Fabiano Motta Toledo**
</br>
---

Este Notebook apresenta o desenvolvimento de um pipeline de aprendizado de máquina para detectar e classificar anomalias em poços de petróleo utilizando o dataset 3W.

A abordagem utiliza um modelo ensemble que, em primeiro lugar, decide se o estado é anômalo (classificação binária: normal vs. anômalo) e, caso seja anômalo, classifica o tipo de evento (rótulos de 1 a 9). O treinamento é realizado com dados de um poço e a validação das métricas é efetuada com dados de outro poço.



Clonagem do repositório contendo o dataset 3W e instalação das bibliotecas necessárias:

In [1]:
!git clone https://github.com/petrobras/3W.git

Cloning into '3W'...
remote: Enumerating objects: 8562, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 8562 (delta 130), reused 115 (delta 114), pack-reused 8369 (from 2)
Receiving objects: 100% (8562/8562), 3.06 GiB | 22.38 MiB/s, done.
Resolving deltas: 100% (2775/2775), done.
Updating files: 100% (2322/2322), done.


In [1]:
!pip install scikeras #para a integração do Keras com o scikit-learn
!pip install PyWavelets #para a transformação wavelet


Importação das bibliotecas necessárias para manipulação dos dados, pré-processamento, construção e avaliação dos modelos:

In [42]:
# Importação das Bibliotecas Necessárias
import os
import glob
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import pywt

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

# Para o modelo LSTM
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scikeras.wrappers import KerasClassifier

import matplotlib.pyplot as plt
%matplotlib inline


2.18.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Funções para Carregamento e Pré-processamento dos Dados

Carregamento dos Dados

Cada arquivo Parquet é carregado e uma coluna extra “instance” (identificando o arquivo) é adicionada para posterior agregação.

In [3]:
def load_well_data(paths):
    dfs = []
    for file in paths:
        df = pd.read_parquet(file)
        df['instance'] = os.path.basename(file)
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)


Definindo os caminhos dos arquivos

In [4]:
# Arquivos do poço 34 (para treinamento e validação)
paths_well34 = [
    "/content/3W/dataset/9/WELL-00034_20190825004401.parquet",
    "/content/3W/dataset/9/WELL-00034_20190831210153.parquet",
    "/content/3W/dataset/9/WELL-00034_20190826050146.parquet",
    "/content/3W/dataset/9/WELL-00034_20190831210153.parquet",
    "/content/3W/dataset/9/WELL-00034_20190901060904.parquet",
    "/content/3W/dataset/9/WELL-00034_20191003180056.parquet"
]

# Arquivos do poço 33 (para teste/deploy)
paths_well33 = [
    "/content/3W/dataset/9/WELL-00033_20190925190420.parquet",
    "/content/3W/dataset/9/WELL-00033_20190926065206.parquet",
    "/content/3W/dataset/9/WELL-00033_20190928030111.parquet",
    "/content/3W/dataset/9/WELL-00033_20190928210207.parquet",
    "/content/3W/dataset/9/WELL-00033_20190930190000.parquet",
    "/content/3W/dataset/9/WELL-00033_20191004020000.parquet"
]


Carregando os dados dos poços:

In [5]:
df_well34 = load_well_data(paths_well34)  # Usado para treinamento/validação
df_well33 = load_well_data(paths_well33)  # Usado para Teste/Deploy

## Definição da Flag de Anomalia

Cria-se a coluna "is_anomaly" a partir da coluna "class". Segundo as regras:

Se "class" for 0, a operação é normal (is_anomaly = 0);

Caso contrário (por exemplo, 9 ou 109), is_anomaly = 1.

In [6]:
df_well34['is_anomaly'] = df_well34['class'].apply(lambda x: 0 if x == 0 else 1)
df_well33['is_anomaly'] = df_well33['class'].apply(lambda x: 0 if x == 0 else 1)

## Agregação dos Dados e Extração de Features:

Para cada instância (arquivo), calcula-se as estatísticas (média, std, mínimo e máximo) das colunas de sensores. Na agregação da coluna `"class"`, utiliza-se o valor máximo, de forma que:

* Se todos os valores forem 0, a instância é normal;
* Se houver pelo menos um 9, o máximo será 9;
* Se houver pelo menos um 109, o máximo será 109.

Essa abordagem também é aplicada para "is_anomaly" (usando o .max()).

In [7]:
def aggregate_features(df, sensor_cols):
    agg_funcs = ['mean', 'std', 'min', 'max']
    agg_df = df.groupby('instance')[sensor_cols].agg(agg_funcs)
    agg_df.columns = ['_'.join(col).strip() for col in agg_df.columns.values]

    classes = df.groupby('instance')['class'].max()
    is_anomaly = df.groupby('instance')['is_anomaly'].max()

    agg_df['class'] = classes
    agg_df['is_anomaly'] = is_anomaly
    return agg_df

In [15]:
# Seleciona as colunas de sensores: todas as colunas numéricas, exceto as de identificação
cols_exclude = ['timestamp', 'instance', 'class', 'is_anomaly']
sensor_cols = [col for col in df_well34.columns if col not in cols_exclude and pd.api.types.is_numeric_dtype(df_well34[col])]


In [14]:
# Agrega os dados para cada poço
agg_well34 = aggregate_features(df_well34, sensor_cols)
agg_well33 = aggregate_features(df_well33, sensor_cols)

In [17]:
features_well34 = agg_well34.drop(['class', 'is_anomaly'], axis=1)
features_well33 = agg_well33.drop(['class', 'is_anomaly'], axis=1)

# Colunas comums nos dois poços
common_cols = features_well34.columns.intersection(features_well33.columns)

# Seleciona as colunas comumns
features_well34 = features_well34[common_cols]
features_well33 = features_well33[common_cols]

# Remover features com variância 0
selector = VarianceThreshold(threshold=0)
features_well34_sel = selector.fit_transform(features_well34)
features_well33_sel = selector.transform(features_well33)
features_selected = features_well34.columns[selector.get_support()]

/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_variance_threshold.py:114: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)
/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_variance_threshold.py:122: RuntimeWarning: All-NaN slice encountered
  self.variances_ = np.nanmin(compare_arr, axis=0)


## Pré-Processamento: Normalização e Redução de Dimensionalidade

Agora, vamos separar as features e os rótulos (usando o target binário `is_anomaly`), em seguida:

* Divisão do conjunto do poço 41 em treino e validação;
* Normalização com `StandardScaler`;
* Redução de dimensionalidade com PCA (mantendo 95% da variância).

In [18]:
agg_well34_clean = agg_well34.dropna(axis=1)
agg_well33_clean = agg_well33.dropna(axis=1)

In [19]:
# Separa features e rótulos. Como "class" e "is_anomaly" são os rótulos, eles são removidos das features.
features_well34 = agg_well34_clean.drop(['class','is_anomaly'], axis=1)
features_well33 = agg_well33_clean.drop(['class','is_anomaly'], axis=1)
target_well34 = agg_well34_clean['is_anomaly']
target_well33 = agg_well33_clean['is_anomaly']

In [21]:
common_cols = features_well34.columns.intersection(features_well33.columns)
features_well34 = features_well34[common_cols]
features_well33 = features_well33[common_cols]

In [22]:
selector = VarianceThreshold(threshold=0)
features_well34_sel = selector.fit_transform(features_well34)
features_well33_sel = selector.transform(features_well33)
features_selected = features_well34.columns[selector.get_support()]

X_well34 = pd.DataFrame(features_well34_sel, columns=features_selected, index=features_well34.index)
X_well33 = pd.DataFrame(features_well33_sel, columns=features_selected, index=features_well33.index)


In [51]:
# Divisão interna dos dados do poço 34: 70% para treino e 30% para validação
X_train, X_val, y_train, y_val = train_test_split(X_well34, target_well34, test_size=0.3, random_state=42, stratify=target_well34)

# Verifica a distribuição das classes no conjunto de treinamento
print("Distribuição em y_train:", np.unique(y_train, return_counts=True))
if len(np.unique(y_train)) < 2:
    print("Atenção: Apenas uma classe em y_train. Inserindo amostra dummy com rótulo 0 (normal).")
    dummy_sample = np.zeros((1, X_train.shape[1]))
    X_train = np.vstack([X_train, dummy_sample])
    y_train = np.append(y_train, 0)

# Adicionado para estratificar o conjunto de dados dentro do StackingClassifier
stack = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42))


Distribuição em y_train: (array([1]), array([3]))
Atenção: Apenas uma classe em y_train. Inserindo amostra dummy com rótulo 0 (normal).


In [52]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_train_full_scaled = scaler.transform(X_well34)
X_test_deploy_scaled = scaler.transform(X_well33)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [53]:
# PCA: reduz a dimensionalidade mantendo 95% da variância
pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_scaled)
X_val_pca   = pca.transform(X_val_scaled)
X_train_full_pca = pca.transform(X_train_full_scaled)
X_test_deploy_pca = pca.transform(X_test_deploy_scaled)

n_features = X_train_pca.shape[1]
print("Número de features após PCA:", n_features)

Número de features após PCA: 3


## Preparação para o Modelo LSTM

Adiciona uma dimensão temporal (timestep único) para que os dados possam ser processados pelo LSTM.

In [54]:
X_train_pca_lstm = X_train_pca.reshape(-1, 1, n_features)
X_val_pca_lstm   = X_val_pca.reshape(-1, 1, n_features)
X_train_full_pca_lstm = X_train_full_pca.reshape(-1, 1, n_features)
X_test_deploy_pca_lstm = X_test_deploy_pca.reshape(-1, 1, n_features)

## Definição dos Modelos e Criação do Ensemble

São definidos três modelos:

* **Random Forest** e **Gradient Boosting**: trabalham com dados 2D.

* **LSTM**: definido para aceitar um input 2D e, em seu primeiro layer, realizar um reshape para (1, n_features).

Os modelos são integrados em um `ensemble via stacking` (meta-classificador: regressão logística).


In [55]:
def create_lstm_model(input_shape):
    # input_shape será do tipo (n_features,)
    from tensorflow.keras.layers import Input, Reshape, Dropout # Importando as camadas de Input, Reshape e Dropout
    model = Sequential()
    # Adiciona uma camada de Input e, em seguida, reestrutura para (1, n_features)
    model.add(Input(shape=input_shape))
    model.add(Reshape((1, input_shape[0])))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [56]:
# Aqui, definimos o input shape como (n_features,) para o LSTM, pois o modelo realizará o reshape internamente.
lstm_clf = KerasClassifier(model=create_lstm_model,
                             model__input_shape=(n_features,),
                             epochs=10, batch_size=16, verbose=0)

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
gb_clf = GradientBoostingClassifier(n_estimators=100, random_state=42)

estimators = [
    ('rf', rf_clf),
    ('gb', gb_clf),
    ('lstm', lstm_clf)
]

stack = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), cv=3)


In [57]:
from collections import Counter
print("Class distribution in y_train:", Counter(y_train))

Class distribution in y_train: Counter({np.int64(1): 3, np.int64(0): 1})


## Treinamento e Avaliação Interna (Poço 34)

Treina o ensemble com os dados de treino do Poço 34 (após PCA) e avaliado no conjunto de validação.

São calculadas as métricas: acurácia, F1-Score, ROC AUC, matriz de confusão, especificidade e FAR.

In [58]:
stack.fit(X_train_pca, y_train)
y_val_pred = stack.predict(X_val_pca)
f1 = f1_score(y_val, y_val_pred)
accuracy = accuracy_score(y_val, y_val_pred)
roc_auc = roc_auc_score(y_val, stack.predict_proba(X_val_pca)[:,1])
cm = confusion_matrix(y_val, y_val_pred)

print("### Métricas de Validação Interna (Poço 34)")
print("Acurácia:", accuracy)
print("F1 Score:", f1)
print("ROC AUC:", roc_auc)
print("Matriz de Confusão:\n", cm)

tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
far = fp / (tn + fp) if (tn + fp) != 0 else 0
print("Especificidade (SPE):", specificity)
print("Taxa de Falsos Alarmes (FAR):", far)

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:1380: RuntimeWarning: Number of classes in training fold (1) does not match total number of classes (2). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


ValueError: y contains 1 class after sample_weight trimmed classes with zero weights, while a minimum of 2 classes are required.